In [14]:
from binance.client import Client 
import pandas as pd  
import os 
from dotenv import load_dotenv
from pathlib import Path
import time
import requests

load_dotenv()
API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")

if not API_KEY or not API_SECRET:
    raise EnvironmentError("API_KEY or API_SECRET not set.")


client = Client(API_KEY, API_SECRET, testnet=True)
client.API_URL = "https://testnet.binance.vision/api"       # ! Make sure system time is correct
account = client.get_account()

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime, timedelta

from binance.client import Client
import yfinance as yf 

from pathlib import Path
from dotenv import load_dotenv


class MarketDataDownloader:
    """
    Class to download historical market data.
    Args:
    - asset: 'Crypto', 'Stock', or 'Forex'
    - ticker: Ticker symbol (e.g., 'BTCUSDT', 'AAPL', 'EURUSD')
    - interval: Data interval (e.g., '1m', '5m', '1h', '1d')
    - lookback: Lookback period (e.g., '1d', '7d', '1m', '1y')
    """

    def __init__(
            self,
            # asset: str,
            # ticker: str,
            interval = '1d',
            lookback = None,
            start = None,
            end = None):
        
        # self.asset = asset
        # self.ticker = ticker
        self.interval = interval

        # Initializing lookback from NOW or a fixed time
        if lookback:
            self.lookback = lookback
            self.start_dt, self.end_dt = self._parse_lookback(self.lookback)
        elif start and end:
            self.start_dt = pd.to_datetime(start)
            self.end_dt = pd.to_datetime(end)
        else:
            raise ValueError('Either lookback or both start and end must be provided')

        # # Initializing asset class and tickers
        # if self.asset is None:
        #     raise ValueError('Asset must be "Crypto", "Stock", or "Forex"')
        # if self.ticker is None:
        #     raise ValueError('Ticker must be provided')
        
        # Initializing interval mapping for Binance or YF
        
        

    def _parse_lookback(self, lookback):
        """
        Converts lookback strings understood by pandas
        (e.g., '1d', '7d', '1m', '1y') into milliseconds
        """
        now = pd.Timestamp.now()                    # TODO check if this is able to do local tz
        if isinstance(lookback, str):
            delta = pd.to_timedelta(lookback)
            start = now - delta
        else:
            raise ValueError('Lookback must be a string like "1d", "7d", "1m", "1y"')
        return start, now


    def _interval_to_ms(self, interval): 
        """
        Converts a pandas-compatible interval string to milliseconds.
        """
        return int(pd.to_timedelta(interval).total_seconds() * 1000)
    

    def _map_interval(self, asset, interval):
        """
        Maps a generic interval string to the specific format required by YF or Binance.
        This method is called in the fetch method after asset and interval are set!!
        """
        interval_map = {
            '1m': {'binance': '1m', 'yahoo': '1m'},
            '5m': {'binance': '5m', 'yahoo': '5m'},
            '15m': {'binance': '15m', 'yahoo': '15m'},
            '30m': {'binance': '30m', 'yahoo': '30m'},
            '1h': {'binance': '1h', 'yahoo': '60m'},
            '2h': {'binance': '2h', 'yahoo': None},
            '4h': {'binance': '4h', 'yahoo': None},
            '1d': {'binance': '1d', 'yahoo': '1d'},
            '1wk': {'binance': '1w', 'yahoo': '1wk'},
            '1mo': {'binance': '1M', 'yahoo': '1mo'}
        }

        if interval not in interval_map:
            raise ValueError(f"Interval {interval} not supported.")
        
        if asset.lower() == 'crypto':
            mapped = interval_map[interval]['binance']
        else:  # stock or forex
            mapped = interval_map[interval]['yahoo']

        if mapped is None:
            raise ValueError(f"Interval '{interval}' is not supported for asset type '{asset_type}'")

        return mapped


    def fetch(
            self, 
            save_path: str = None,
            asset: str = None,
            ticker: str = None):
        """
        Main fetching method for respective asset class.
        It calls the respective fetch methods.
        Args:
        - save_path: Optional path to save the fetched data as CSV.
        - asset: 'Crypto', 'Stock', or 'Forex'
        - ticker: Ticker symbol (e.g., 'BTCUSDT', 'AAPL', 'EUR/USD')
        """
        self.asset = asset
        self.ticker = ticker

        self.interval = self._map_interval(self.asset, self.interval)
        
        if self.asset.lower() == 'crypto':
            data = self._fetch_crypto()
        elif self.asset.lower() == 'stock':
            data = self._fetch_stock()
        elif self.asset.lower() == 'forex':
            data = self._fetch_forex()
        else:
            raise ValueError('Asset must be "Crypto", "Stock", or "Forex"')

        self.data = data

        if save_path:
            self.data.to_csv(save_path)
            print("Data saved to", save_path)
        else:
            print("No save path provided, data will not be saved to disk.")

        print(f"Fetched {len(data)} rows of data.")
        return data


    def _fetch_crypto(self):
        """
        Method to fetch crypto data from Binance.
        Binance API limits to 1000 data points per request, so we need to roll over multiple requests
        """
        interval_ms = self._interval_to_ms(self.interval)
        start_ms = int(self.start_dt.value // 10**6)
        end_ms = int(self.end_dt.value // 10**6)
        current_start = start_ms
        all_dfs = []
        limit = 1000        # Binance max limit, if premium account maybe we can change this later

        while current_start < end_ms:
            params = {
                'symbol': self.ticker, 
                'interval': self.interval,
                'startTime': current_start,
                'limit': limit
            }
            r = requests.get("https://api.binance.com/api/v3/klines", params=params)
            r.raise_for_status()
            data = r.json()
            if not data:
                break

            df = pd.DataFrame(data, columns=[
                'timestamp', 'open', 'high', 'low', 'close', 'volume',
                'close_time', 'quote_asset_volume', 'trades',
                'taker_buy_base', 'taker_buy_quote', 'ignore'
            ])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            numerics = ['open', 'high', 'low', 'close', 'volume']
            df[numerics] = df[numerics].apply(pd.to_numeric, axis=1)
            df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

            all_dfs.append(df)
            
            # Advance to next candle after the last returned
            last_ts_ms = int(df['timestamp'].iloc[-1].value // 10**6)
            current_start = last_ts_ms + interval_ms

            # Avoid spamming the API
            time.sleep(0.5)

            # If returned fewer than limit, probably reached end range
            if len(data) < limit:
                break

        if not all_dfs:
            return pd.DataFrame(columns=['timestamp','open','high','low','close','volume'])
        
        result = pd.concat(all_dfs, ignore_index=True)
        result = result[result['timestamp'] <= pd.to_datetime(self.end_dt)]
        result = result.drop_duplicates(subset=['timestamp']).reset_index(drop=True)
        return result


    def _fetch_stock(self):
        df = yf.download(self.ticker, start=self.start_dt, end=self.end_dt, interval=self.interval)
        if df.empty:
            print("No data fetched from Yahoo Finance.")
            return df
        df = df.reset_index()
        df.rename(columns = {
            'Date': 'timestamp',
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        }, inplace = True)
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        return df
    
    def _fetch_forex(self):
        self.ticker = self.ticker.replace('/', '') + "=X"
        return self._fetch_stock() # Can use YF to fetch Forex as well

In [63]:
loader = MarketDataDownloader(interval='1h', start = "2023-01-01", end = "2024-01-01")
data = loader.fetch()

AttributeError: 'MarketDataDownloader' object has no attribute 'asset'

In [59]:
loader = MarketDataDownloader(asset='Forex', ticker='USD/EUR', interval='1h',
                              start='2022-01-01', end='2022-12-31')
data = loader.fetch()

C:\Users\script.pc #93\AppData\Local\Temp\ipykernel_26976\2287668856.py:147: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(self.ticker, start=self.start_dt, end=self.end_dt, interval=self.interval)
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['USDEUR=X']: YFPricesMissingError('possibly delisted; no price data found  (1h 2022-01-01 00:00:00 -> 2022-12-31 00:00:00) (Yahoo error = "1h data not available for startTime=1640995200 and endTime=1672444800. The requested range must be within the last 730 days.")')


No data fetched from Yahoo Finance.
No save path provided, data will not be saved to disk.
Fetched 0 rows of data.


In [ ]:
def interval_to_ms(interval_str):
    return int(pd.to_timedelta(interval_str).total_seconds() * 1000) # pandas understands strings like "1h", "1m", "1d"

def download_klines_chunks(symbol, interval, start_dt, end_dt, out_csv,
                           limit=1000, pause=0.5, append=False):
    interval_ms = interval_to_ms(interval)
    start_ms = int(pd.to_datetime(start_dt).value // 10**6)  # ns -> ms
    end_ms = int(pd.to_datetime(end_dt).value // 10**6)

    all_dfs = []
    current_start = start_ms

    while current_start < end_ms:
        params = {
            "symbol": symbol,
            "interval": interval,
            "startTime": current_start,
            "limit": limit
        }
        r = requests.get("https://api.binance.com/api/v3/klines", params=params)
        r.raise_for_status()
        data = r.json()
        if not data:
            break

        df = pd.DataFrame(data, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_asset_volume', 'trades',
            'taker_buy_base', 'taker_buy_quote', 'ignore'
        ])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        numerics = ['open', 'high', 'low', 'close', 'volume']
        df[numerics] = df[numerics].apply(pd.to_numeric, axis=1)
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

        all_dfs.append(df)

        # Advance to next candle after the last returned
        last_ts_ms = int(df['timestamp'].iloc[-1].value // 10**6)
        current_start = last_ts_ms + interval_ms

        # avoid spamming the API
        time.sleep(pause)

        # if returned fewer than limit, probably reached end range
        if len(data) < limit:
            break

    if not all_dfs:
        return pd.DataFrame(columns=['timestamp','open','high','low','close','volume'])

    result = pd.concat(all_dfs, ignore_index=True)

    # trim to requested end_dt and remove duplicates
    result = result[result['timestamp'] <= pd.to_datetime(end_dt)]
    result = result.drop_duplicates(subset=['timestamp']).reset_index(drop=True)

    # ensure output directory exists and write CSV (append or overwrite)
    out_path = Path(out_csv)
    # if provided a relative path, resolve relative to the notebook's cwd (notebooks folder)
    if not out_path.is_absolute():
        out_path = (Path.cwd() / out_path).resolve()

    out_path.parent.mkdir(parents=True, exist_ok=True)

    # show the resolved absolute path so you know where file is written
    print("Writing CSV to:", out_path)

    if append:
        result.to_csv(out_path, mode='a', header=not out_path.exists(), index=False)
    else:
        result.to_csv(out_path, index=False)

    return result

# 2020 January 1 to 2024 December 31
start = pd.Timestamp("2020-01-01")
end   = pd.Timestamp("2024-12-31 23:59:59")

# save into project data/raw_data relative to the notebooks folder
out_file = Path('..') / 'data' / 'raw_data' / 'BTCUSDT_1h.csv'
print('Kernel cwd:', Path.cwd())
print('Resolved out_file:', (Path.cwd() / out_file).resolve())

# call downloader with the relative path 
df = download_klines_chunks('BTCUSDT', '1h', start, end, out_csv=str(out_file))

display(df)

Kernel cwd: d:\pricing-comparison\notebooks
Resolved out_file: D:\pricing-comparison\data\raw_data\BTCUSDT_1h.csv
Writing CSV to: D:\pricing-comparison\data\raw_data\BTCUSDT_1h.csv
Writing CSV to: D:\pricing-comparison\data\raw_data\BTCUSDT_1h.csv


,timestamp,open,high,low,close,volume
0,2020-01-01 00:00:00,7195.24,7196.25,7175.46,7177.02,511.814901
1,2020-01-01 01:00:00,7176.47,7230.00,7175.71,7216.27,883.052603
2,2020-01-01 02:00:00,7215.52,7244.87,7211.41,7242.85,655.156809
3,2020-01-01 03:00:00,7242.66,7245.00,7220.00,7225.01,783.724867
4,2020-01-01 04:00:00,7225.00,7230.00,7215.03,7217.27,467.812578
...,...,...,...,...,...,...
43811,2024-12-31 19:00:00,93875.69,94290.91,93712.45,94166.88,462.793420
43812,2024-12-31 20:00:00,94166.88,94222.50,93450.17,93564.04,733.041470
43813,2024-12-31 21:00:00,93564.01,93964.15,93504.67,93899.68,337.527150
43814,2024-12-31 22:00:00,93899.67,93899.67,93375.58,93488.84,315.532720


In [ ]:
# def get_klines_no_auth(symbol, interval, lookback_days):
#     end_time = int(time.time() * 1000)                              # Current time in milliseconds
#     start_time = end_time - (lookback_days * 24 * 60 * 60 * 1000)   # Convert lookback days to ms

#     start_dt = pd.to_datetime(start_time, unit='ms')
#     end_dt = pd.to_datetime(end_time, unit='ms')
#     print(f"Fetching data from {start_dt} to {end_dt}")
    
#     url = "https://api.binance.com/api/v3/klines"
#     params = {
#         "symbol": symbol,                               
#         "interval": interval,                           
#         "startTime": start_time,                        # Start timestamp in ms
#         "endTime": end_time,                            # End timestamp in ms
#         "limit": 1000                                   # Max candles per request (Binance's limit)
#     }
    
#     response = requests.get(url, params=params)
#     data = response.json()
    
#     # Convert to DataFrame
#     df = pd.DataFrame(data, columns=[
#         'timestamp', 'open', 'high', 'low', 'close', 'volume',
#         'close_time', 'quote_asset_volume', 'trades',
#         'taker_buy_base', 'taker_buy_quote', 'ignore'
#     ])
#     df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
#     tumeric_col = ['open', 'high', 'low', 'close', 'volume']
#     df[tumeric_col] = df[tumeric_col].apply(pd.to_numeric, axis=1)
#     return df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# btc_hourly = get_klines_no_auth("BTCUSDT", "1h", 60)
# display(btc_hourly)

Fetching data from 2025-06-27 16:43:48.227000 to 2025-08-26 16:43:48.227000


,timestamp,open,high,low,close,volume
0,2025-06-27 17:00:00,107480.10,107497.16,106477.34,106875.75,501.23090
1,2025-06-27 18:00:00,106875.75,107000.00,106446.45,106704.60,440.83658
2,2025-06-27 19:00:00,106704.60,107020.66,106594.10,106801.02,366.69076
3,2025-06-27 20:00:00,106801.02,107130.36,106794.33,107119.50,254.52948
4,2025-06-27 21:00:00,107119.50,107277.99,107038.54,107052.01,231.16317
...,...,...,...,...,...,...
995,2025-08-08 04:00:00,116737.34,116869.49,116666.00,116750.01,325.84463
996,2025-08-08 05:00:00,116750.01,116750.01,116505.00,116530.63,346.36157
997,2025-08-08 06:00:00,116530.64,116763.44,116401.00,116740.01,336.48270
998,2025-08-08 07:00:00,116740.01,116973.30,116557.54,116817.19,447.58601
